In [1]:
""" Convert RDS table to csv files on S3 and GCS
-------------------------------------------------------------------------------

update 2020/02/10 update output 11-12 input 8-9

Author: Rutger Hofste
Date: 20180712
Kernel: python35
Docker: rutgerhofste/gisdocker:ubuntu16.04
"""

# imports
import re
import os
import numpy as np
import pandas as pd
from retrying import retry
from datetime import timedelta
from sqlalchemy import *
pd.set_option('display.max_columns', 500)
import multiprocessing

SCRIPT_NAME = 'Y2018M07D17_RH_RDS_To_S3_V02'
OUTPUT_VERSION = 13

TESTING = 0

DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
DATABASE_NAME = "database01"

INPUT_TABLE_NAME = "y2018m07d30_rh_coalesce_columns_v01_v09"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
gcs_output_path = "gs://aqueduct30_v01/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)

print("Input Table: " , INPUT_TABLE_NAME,
      "\nOutput ec2: ", ec2_output_path,
      "\nOutput s3: " , s3_output_path,
      "\nOutput gcs: ", gcs_output_path)

Input Table:  y2018m07d30_rh_coalesce_columns_v01_v09 
Output ec2:  /volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13/ 
Output s3:  s3://wri-projects/Aqueduct30/processData/Y2018M07D17_RH_RDS_To_S3_V02/output_V13/ 
Output gcs:  gs://aqueduct30_v01/Y2018M07D17_RH_RDS_To_S3_V02/output_V13/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2020M02D14 UTC 11:50


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}

rm: cannot remove '/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13/': No such file or directory


In [4]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,DATABASE_ENDPOINT,DATABASE_NAME))

In [5]:
cpu_count = multiprocessing.cpu_count()
print("Power to the maxxx:", cpu_count)

Power to the maxxx: 16


In [6]:
sql = "SELECT DISTINCT pfafid_30spfaf06 FROM {} ORDER BY pfafid_30spfaf06".format(INPUT_TABLE_NAME)

In [7]:
df = pd.read_sql(sql,engine)

In [8]:
df.shape

(16385, 1)

In [9]:
df.head()

,pfafid_30spfaf06
0,111011
1,111012
2,111013
3,111014
4,111015


In [10]:
if TESTING:
    df = df[0:10]

In [11]:
df_split = np.array_split(df, cpu_count*100)

In [12]:
def basin_to_csv(df):
    for index, row in df.iterrows():
        pfafid = row["pfafid_30spfaf06"]
        sql = "SELECT * FROM {} WHERE pfafid_30spfaf06 = {}".format(INPUT_TABLE_NAME,pfafid)
        df_basin = pd.read_sql(sql,engine)
        now = datetime.datetime.now()
        df_basin["processed_timestamp"] = pd.Timestamp(now)  
        output_file_name = "{}_V{:02.0f}.csv".format(pfafid,OUTPUT_VERSION)
        output_file_path = "{}/{}".format(ec2_output_path,output_file_name)
        # Added on Y2018M07D30, convert to numeric type if possible
        df_basin2 = df_basin.apply(pd.to_numeric,errors='ignore')
        df_basin2.to_csv(output_file_path)
        print(output_file_path)

In [13]:
# cleared output to save space
p= multiprocessing.Pool()
results_buffered = p.map(basin_to_csv,df_split)
p.close()
p.join()

Exception during reset or similar
Traceback (most recent call last):
  File "/opt/anaconda3/envs/python35/lib/python3.5/site-packages/sqlalchemy/pool.py", line 709, in _finalize_fairy
    fairy._reset(pool)
  File "/opt/anaconda3/envs/python35/lib/python3.5/site-packages/sqlalchemy/pool.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "/opt/anaconda3/envs/python35/lib/python3.5/site-packages/sqlalchemy/engine/default.py", line 457, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: SSL error: sslv3 alert bad record mac



/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111011_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//155562_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//127443_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115801_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//132895_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//142625_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//145102_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//181976_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//157807_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//172497_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//232144_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//223940_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//153021_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//122784_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//158115_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111019_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//224002_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115808_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//122810_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//145109_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//181993_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//172524_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//232160_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//162300_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//131909_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//153028_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//132941_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//142634_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//116005_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//224009_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//172541_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//145207_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//162444_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//184000_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//232240_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//132948_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//142642_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111046_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//132170_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//122846_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//155580_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//153035_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117101_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//155617_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//225117_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//132956_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111081_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//132224_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//162461_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//232302_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//211016_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//127485_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//122854_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//142651_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//145404_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//158211_V13.csv
/volumes/data/Y2018M

...

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911107_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//722805_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//831959_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//753303_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//772342_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//662807_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//712507_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//642461_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911108_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//726014_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//742987_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//812109_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//822479_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//831960_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//662905_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//742995_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//812207_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//822496_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//742450_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//832104_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911220_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//723030_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//782703_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//652309_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//726040_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//642469_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//772350_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//753402_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//782801_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//743005_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//723074_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//772403_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//726075_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//642477_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//664022_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//812306_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//753409_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//712900_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911268_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//822515_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//832421_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//742486_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//751203_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//782809_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911288_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//726084_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//664041_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//772601_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//723111_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//713225_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//832429_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//753508_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//812450_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//822542_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//742494_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911289_V13.csv
/volumes/data/Y2018M

/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//751402_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//731120_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//772700_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//782980_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//812478_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//832449_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//753626_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//723120_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//713270_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//822560_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//751403_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//772801_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//782991_V13.csv
/volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//812479_V13.csv
/volumes/data/Y2018M

In [14]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive --quiet

In [15]:
# cleared output to save space
!gsutil -m cp \
{ec2_output_path}/*.csv \
{gcs_output_path}

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111011_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111012_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111013_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111016_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111014_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111015_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111017_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111018_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//111020_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112120_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112140_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112018_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112180_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112190_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112200_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112400_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112300_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//112600_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114445_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114446_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114447_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114448_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114449_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114450_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114460_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114470_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//114480_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115426_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115427_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115428_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115429_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115430_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115440_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115451_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115452_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//115453_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117105_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117108_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117106_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117201_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117202_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117109_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117203_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117204_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//117205_V13.csv [Content-Type=text/

...

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863003_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863004_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863005_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863006_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863007_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863008_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863009_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863011_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//863012_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911405_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911406_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911407_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911408_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911409_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911500_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911510_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911520_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//911530_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912208_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912209_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912300_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912400_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912510_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912520_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912530_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912540_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//912551_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913479_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913480_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913490_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913500_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913510_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913520_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913530_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913540_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//913550_V13.csv [Content-Type=text/

Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914506_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914507_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914508_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914509_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914600_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914700_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914701_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914702_V13.csv [Content-Type=text/csv]...
Copying file:///volumes/data/Y2018M07D17_RH_RDS_To_S3_V02/output_V13//914703_V13.csv [Content-Type=text/

In [16]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:30:37.392875


Previous runs:  
0:42:01.416597  
0:42:14.565295  
0:31:40.728932  
0:23:35.132388  
0:30:37.392875

